In [112]:
import json
import codecs
from pyspark.sql import SparkSession
from collections import defaultdict
import redis


In [113]:
feature = {"type":"uType", "cid":"categoryId", "pid":"parentCategoryId",
        "ctr":"reItemCtr","collect1":"collectInterval", "add1":"addcartInterval", "tag":"kind",
        "uid":"hashUid", "province_city":"uCity_province", "tag_city":"kind_province", 
        "tag_grade":"kind_categoryGrade", "tag_uid":"kind_hashUid", "province":"uCity",
         "sale":"sellcntInterval","ctr_sale":"reItemCtr_sellcntInterval","iftag":"preferenceKind","ifcid":"preferenceCategory"
        }

def model():
    spark = SparkSession.builder.appName("lr_sample_train").getOrCreate()
    craftsman_city = spark.sql("SELECT new_city, row_number() OVER(ORDER BY new_city) as num from (SELECT DISTINCT new_city from songwt.city_normalize) t").collect()
    _dict = dict()
    for row in craftsman_city:
        _dict[row.new_city] = row.num
        
    lr_model = codecs.open("model", "wb")
    for fn in codecs.open("best_model", "rb"):
        ln = fn.strip().decode("utf8")
        lnn = ln.split("\t")
        name = lnn[0]
        value = lnn[1]
        weight = lnn[2]
        
        if weight == 0.0 or name == "(INTERCEPT)":
            continue
        if len(name.split("_")) == 2:
            new_name = feature[name]
            lr_name = new_name
        
            if new_name.find("province") != -1 and value.split("_")[1] != "null":
                
                province = _dict[value.split("_")[1]]

                lr_term = value.split("_")[0]+"_"+str(province)
            
            else:
                lr_term = value
        else:
            lr_name = feature[name]
            lr_term = value
        #print(lr_name)
        lr_model.write(lr_name.encode("utf8"))
        lr_model.write(",".encode("utf8"))
        lr_model.write(lr_term.encode("utf8"))
        lr_model.write(",".encode("utf8"))
        lr_model.write(weight.encode("utf8"))
        lr_model.write("\n".encode("utf8"))
    
    print("finished")
    


In [114]:
def getctr(ctr):
    if ctr <= 0.075:
        ctr_level = 0
    elif ctr > 0.075 and ctr <= 0.08730159:
        ctr_level = 1
    elif ctr > 0.08730159 and ctr <= 0.09428571:
        ctr_level = 2
    elif ctr > 0.09428571 and ctr <= 0.09803922:
        ctr_level = 3
    elif ctr > 0.09803922 and ctr <= 0.1015625:
        ctr_level = 4
    elif ctr > 0.1015625 and ctr <=  0.11111111:
        ctr_level = 5
    elif ctr > 0.11111111 and ctr <= 0.11956522:
        ctr_level = 6
    elif ctr > 0.11956522 and ctr <= 0.13461538:
        ctr_level = 7
    elif ctr > 0.13461538 and ctr <= 0.15957447:
        ctr_level = 8
    else:
        ctr_level = 9
    return ctr_level

def getcollect(collect):
    if collect == 0:
        collect_level = 0
    elif collect == 1:
        collect_level = 1
    elif collect > 1 and collect <= 3:
        collect_level = 2
    elif collect > 3 and collect <= 16:
        collect_level = 3
    else:
        collect_level = 4
    return collect_level

def getaddcart(addcart):
    if addcart == 0:
        addcart_level = 0
    elif addcart == 1:
        addcart_level = 1
    elif addcart == 2:
        addcart_level = 2
    elif addcart == 3:
        addcart_level = 3
    else:
        addcart_level = 4
    return addcart_level

def getsale(sale):
    if sale == 0:
        sale_level = 0
    elif sale == 1:
        sale_level = 1
    elif sale == 2:
        sale_level = 2
    elif sale == 2:
        sale_level = 3
    else:
        sale_level = 4
    return sale_level

def sort(uid, iid):
    features = defaultdict(dict)
    for fn in codecs.open("model", "r"):
        ln = fn.strip()
        lnn = ln.split(",")
        name = lnn[0]
        term = lnn[1]
        value = lnn[2]
        features[name][term] = float(value)
    print(list(features.keys()))
    
    r = redis.Redis(host='172.16.4.25', password='8ed4d756495de0180ec2e1b72e0b894e', db=15)
    if not r.exists("if_%s" % str(iid)):
        return "商品id:%d,不存在" % iid
    
    
    preference_categories = []
    preference_kinds = []
    city = "others"
    utype = 0
    if r.exists("uf_%s" % uid):
        uid_value = json.loads(r.get("uf_%s" % uid))
        prefercid = uid_value['preference_categories']
        for elem in prefercid:
            preference_categories.append(list(elem.keys())[0])
        city = uid_value['city']
        utype = uid_value['type']
        preferkind = uid_value['preference_kinds']
        for elem in preferkind:
            preference_kinds.append(list(elem.keys())[0])
    
    print(city, utype, preference_categories, preference_kinds)
    
    iid_value = json.loads(r.get("if_%s" % str(iid)))
    ctr = iid_value['ctr']
    ctr_level = getctr(float(ctr))
    
    
    cart = iid_value['addcart']
    cart_level = getaddcart(int(cart))
    
    
    collect = iid_value['collect']
    collect_level = getcollect(int(collect))

    
    sale = iid_value['sale']
    sale_level = getsale(int(sale))
    
    sellerid = iid_value['uid']
    kind = iid_value['kind']
    pid = iid_value['parentId']
    cid = iid_value['categoryId']
    hashuid = iid_value['hashuid']
    grade = iid_value['grade']
    print("grade is:",grade)
    province = iid_value['province']
    print("province is:", province)
    
    if kind in preference_kinds:
        iftag = 1
    else:
        iftag = 0
    
    if cid in preference_categories:
        ifcid = 1
    else:
        ifcid = 0
        
    #用户特征
    type_weight = features["uType"].get(str(utype), 0)
    print("utype is:", utype, type_weight)
    
    iftag_weight = features["preferenceKind"].get(str(iftag), 0)
    print("iftag is:", iftag, iftag_weight)
    
    
    ifcid_weight = features["preferenceCategory"].get(str(ifcid), 0)
    print("ifcid is:", ifcid, ifcid_weight)
    
    #商品特征
    cid_weight = features["categoryId"].get(str(cid), 0)
    print("categoryId is:", cid, cid_weight)
    
    pid_weight = features["parentCategoryId"].get(str(pid), 0)
    print("parentCategoryId is:", pid, pid_weight)
    
    ctr_weight = features["reItemCtr"].get(str(ctr_level), 0)
    print("reItemCtr is:", ctr, ctr_level, ctr_weight)
    
    hashuid_weight = features["hashUid"].get(str(hashuid), 0)
    print("hashuid is:", hashuid, hashuid_weight)
    
    kind_weight = features["kind"].get(str(kind), 0)
    print("kind is:", kind, kind_weight)
    
    collect_weight = features["collectInterval"].get(str(collect_level), 0)
    print("collectInterval is:", collect, collect_level, collect_weight)
    
    cart_weight = features["addcartInterval"].get(str(cart_level), 0)
    print("addcartInterval is:", cart, cart_level, cart_weight)
    
    sale_weight = features["sellcntInterval"].get(str(sale_level), 0)
    print("sellcntInterval is:", sale, sale_level, sale_weight)
    
    weight1 = type_weight+iftag_weight+ifcid_weight+cid_weight+pid_weight+ctr_weight+hashuid_weight+kind_weight+collect_weight+cart_weight+sale_weight
    print(weight1)
    
    
    #交叉特征
    ucity_province = features["uCity_province"].get(city+"_"+str(province), 0)
    print("uCity_province is:", city+"_"+str(province), ucity_province)
    
    kind_categoryGrade = features["kind_categoryGrade"].get(str(kind)+"_"+str(grade), 0)
    print("kind_categoryGrade is:", str(kind)+"_"+str(grade), kind_categoryGrade)
    
    kind_province = features["kind_province"].get(str(kind)+"_"+str(province), 0)
    print("kind_province is:", str(kind)+"_"+str(province), kind_province)
    
    reItemCtr_sellcntInterval = features["reItemCtr_sellcntInterval"].get(str(ctr_level)+"_"+str(sale_level), 0)
    print("reItemCtr_sellcntInterval is:", str(ctr_level)+"_"+str(sale_level), reItemCtr_sellcntInterval)
    
    kind_hashUid = features["kind_hashUid"].get(str(kind)+"_"+str(hashuid), 0)
    print("kind_hashUid is:",str(kind)+"_"+str(hashuid), kind_hashUid)
    
    weight2 = ucity_province+kind_categoryGrade+kind_province+reItemCtr_sellcntInterval+kind_hashUid
    
    weight = type_weight+iftag_weight+ifcid_weight+cid_weight+pid_weight+ctr_weight+hashuid_weight+kind_weight+collect_weight+cart_weight+sale_weight+ucity_province+kind_categoryGrade+kind_province+reItemCtr_sellcntInterval+kind_hashUid
    
    print(uid, iid, weight1, weight2, weight)
        
        
    
    
    
                                    
        
    
    
    
        
        
        

In [115]:
if __name__ == "__main__":
    sort("1443928", 77)

['uCity_province', 'kind_hashUid', 'categoryId', 'hashUid', 'reItemCtr', 'kind_categoryGrade', 'kind_province', 'parentCategoryId', 'preferenceKind', 'reItemCtr_sellcntInterval', 'kind', 'preferenceCategory', 'sellcntInterval', 'collectInterval', 'addcartInterval', 'uType']
浙江省 0 [] ['36', '1', '23']
grade is: 3
province is: 77
utype is: 0 -0.09567258615071735
iftag is: 0 -0.348477070719737
ifcid is: 0 -0.13889951635445122
categoryId is: 254 0.0
parentCategoryId is: 252 0.0
reItemCtr is: 0.1 4 -0.03341791810449746
hashuid is: 50409 0.0
kind is: 18 -0.09327344924473477
collectInterval is: 1 1 -0.015584600696713446
addcartInterval is: 0 0 -0.054120864687110116
sellcntInterval is: 1 1 -0.02909720176489633
-0.8085432077228576
uCity_province is: 浙江省_77 0.0
kind_categoryGrade is: 18_3 -0.0074762466086146385
kind_province is: 18_77 -0.06461264375026854
reItemCtr_sellcntInterval is: 4_1 0.0
kind_hashUid is: 18_50409 0.0
1443928 77 -0.8085432077228576 -0.07208889035888318 -0.8806320980817408
